In [1]:
import re

import pandas as pd

## Jigsaw Toxic Comments

In [2]:
jigsaw_train = pd.read_csv("./data/raw/jigsaw/train.csv")
jigsaw_test = pd.read_csv("./data/raw/jigsaw/test.csv")
jigsaw_test_labels = pd.read_csv("./data/raw/jigsaw/test_labels.csv")

In [3]:
jigsaw_test = pd.merge(jigsaw_test, jigsaw_test_labels, on="id", how="inner", validate="1:1")

In [4]:
def clean_prompt(prompt):
    prompt = re.sub("\\n", " ", prompt)
    prompt = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", prompt)
    prompt = re.sub("\[\[.*\]", "", prompt)
    return prompt

In [5]:
jigsaw_df = pd.concat([jigsaw_train, jigsaw_test]).reset_index(drop=True)
jigsaw_normal = jigsaw_df.loc[(jigsaw_df.iloc[:, -6:] == 0).all(axis=1)]
jigsaw_normal.loc[:, "comment_text"] = jigsaw_normal.loc[:, "comment_text"].apply(clean_prompt)
jigsaw_normal_prompts = jigsaw_normal["comment_text"].tolist()

## Anthropic HH-RLHF

In [6]:
anthropic_train_helpful = pd.read_json("./data/raw/anthropic/helpful_base/train.jsonl", lines=True)
anthropic_test_helpful = pd.read_json("./data/raw/anthropic/helpful_base/test.jsonl", lines=True)

anthropic_train_helpful_online = pd.read_json("./data/raw/anthropic/helpful_online/train.jsonl", lines=True)
anthropic_test_helpful_online = pd.read_json("./data/raw/anthropic/helpful_online/test.jsonl", lines=True)

In [7]:
anthropic_df = pd.concat([anthropic_train_helpful, anthropic_test_helpful, anthropic_train_helpful_online, anthropic_test_helpful_online]).reset_index(drop=True)
anthropic_df["chosen"] = anthropic_df["chosen"].apply(lambda x: " ".join(x.split("\n\n")[1].split()[1:]))
anthropic_df = anthropic_df.drop_duplicates(subset=["chosen"]).reset_index(drop=True)

In [8]:
anthropic_normal_prompts = anthropic_df["chosen"].tolist()

In [9]:
normal_prompts = pd.DataFrame({"prompt": jigsaw_normal_prompts + anthropic_normal_prompts, "category": ["jigsaw"] * len(jigsaw_normal_prompts) + ["anthropic"] * len(anthropic_normal_prompts)})
normal_prompts = normal_prompts.sample(frac=1).reset_index(drop=True)
normal_prompts.to_csv("./data/processed/normal_prompts.csv", index=False)

In [10]:
normal_prompts

,prompt,category
0,"Indeed, I noticed the hand trembling as well.",jigsaw
1,""" :The copyright is disputable? What's that ...",jigsaw
2,"Notable per WP:VICTIM & falls under WP:BIO1E, ...",jigsaw
3,(from my talk page) Looks like this has been s...,jigsaw
4,""":Hi Steve, :# Noted on your comments about ...",jigsaw
...,...,...
226021,What's a riddle?,anthropic
226022,""" The David Blaikie piece mentions that Will ...",jigsaw
226023,if that is what it meant the last version was ...,jigsaw
226024,"""== David's old JavaScript repository == No...",jigsaw
